In [2]:
%load_ext autoreload
%autoreload 2

here the objective is to fit the logit graph model to multiple different graph domains and then visualize the original vs fitted graphs

In [3]:

import sys
import os
sys.path.append('../')

#Graph imports
import src.graph as graph
import src.logit_estimator as estimator
import src.utils as utils
import src.model_selection as model_selection
import src.gic as gic
import src.param_estimator as pe
import src.graph as graph
import src.model_selection as ms

# usual imports
import matplotlib.pyplot as plt
import pickle
import math
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import gaussian_kde
import numpy as np
import pandas as pd
import seaborn as sns
import gc
import random
import networkx as nx
from numpy import errstate

from IPython.display import display
from pyvis.network import Network
from mpl_toolkits.axes_grid1 import make_axes_locatable


In [4]:
FOLDER_OUTPUT = f'runs/fitted_graphs_comparison_twitter1'
out_files = os.listdir(FOLDER_OUTPUT)
out_files = sorted(out_files, key=lambda x: os.path.getctime(os.path.join(FOLDER_OUTPUT, x)), reverse=True)

out_files

['comparators_175529494.pkl',
 'comparators_77404713.pkl',
 'comparators_15208246.pkl',
 'comparators_926981.pkl',
 'comparators_17658786.pkl',
 'comparators_175553601.pkl',
 'comparators_227650565.pkl',
 'comparators_33097148.pkl',
 'comparators_14869783.pkl',
 'comparators_31949053.pkl',
 'comparators_5539522.pkl',
 'comparators_19064433.pkl',
 'comparators_68732646.pkl',
 'comparators_14807093.pkl',
 'comparators_46160500.pkl',
 'comparators_17922109.pkl',
 'comparators_255790981.pkl',
 'comparators_15012486.pkl',
 'comparators_27853708.pkl',
 'comparators_6351572.pkl',
 'comparators_18630326.pkl',
 'comparators_16726878.pkl',
 'comparators_22915745.pkl',
 'comparators_252415267.pkl',
 'comparators_18951737.pkl',
 'comparators_64931560.pkl',
 'comparators_37103386.pkl',
 'comparators_51694885.pkl',
 'comparators_14836915.pkl',
 'comparators_3245291.pkl',
 'comparators_85716069.pkl',
 'comparators_17503591.pkl',
 'comparators_216843160.pkl',
 'comparators_151495845.pkl',
 'comparator

In [5]:
import pickle
#pick first id from out_files
id = out_files[0].split('_')[1]

with open(f'{FOLDER_OUTPUT}/comparators_{id}', 'rb') as f:
    comparators = pickle.load(f)


In [6]:
len(comparators)

442

In [7]:
dfs = []
for i in range(len(comparators)):
    dfs.append(comparators[i].summary_df)

In [8]:
dfs[1]

,graph_filename,model,gic_value,param,fit_success,nodes,edges,density,avg_clustering,avg_path_length,diameter,assortativity,num_components,largest_component_size
0,23503181.edges,Original,NaN,N/A,True,101,1455,0.288119,0.612132,1.786733,3,-0.173453,1,101
1,23503181.edges,LG,0.795880,"d=1, sigma=-5.4724",True,101,1506,0.298218,0.301566,1.701782,2,-0.027576,1,101
2,23503181.edges,BA,0.594682,5.0,True,101,480,0.095050,0.190592,2.227129,4,-0.097471,1,101
3,23503181.edges,ER,0.712607,0.2,True,101,994,0.196832,0.196234,1.819010,3,-0.020593,1,101
4,23503181.edges,WS,1.359712,20.2,True,101,1414,0.280000,0.267722,1.720000,2,-0.022361,1,101
5,23503181.edges,GRG,4.282978,1.0,True,101,4942,0.978614,0.981219,1.021386,2,-0.053043,1,101


In [9]:
pd.concat(dfs)

,graph_filename,model,gic_value,param,fit_success,nodes,edges,density,avg_clustering,avg_path_length,diameter,assortativity,num_components,largest_component_size
0,134208594.edges,Original,NaN,N/A,True,100,1677,0.338788,0.612483,1.692929,3,-0.101428,1,100
1,134208594.edges,LG,0.215938,"d=2, sigma=-51.9010",True,100,1672,0.337778,0.340221,1.662222,2,0.010013,1,100
2,134208594.edges,ER,0.408830,0.2,True,100,1008,0.203636,0.203689,1.807273,3,-0.088399,1,100
3,134208594.edges,BA,0.553002,5.0,True,100,475,0.095960,0.179941,2.224646,4,-0.114091,1,100
4,134208594.edges,WS,1.485660,20.178889,True,100,1700,0.343434,0.334949,1.656566,2,-0.028464,1,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,175529494.edges,LG,0.924148,"d=0, sigma=-4.9933",True,202,1198,0.059012,0.059454,2.408009,4,-0.044214,1,202
2,175529494.edges,WS,0.793148,20.2,True,202,1212,0.059701,0.056012,2.399685,4,-0.056745,1,202
3,175529494.edges,ER,0.931616,0.073333,True,202,1475,0.072657,0.068861,2.247771,4,-0.011889,1,202
4,175529494.edges,BA,0.947549,5.0,True,202,985,0.048520,0.112247,2.493572,4,-0.055922,1,202


In [10]:
# Create a DataFrame with GIC values for each model and graph
gic_df = pd.DataFrame()

for df in dfs:
    graph_name = df['graph_filename'].iloc[0]
    # Get GIC values for each model (excluding Original which has NaN)
    model_gics = df[df['model'] != 'Original'].set_index('model')['gic_value']
    gic_df[graph_name] = model_gics

# Transpose so graphs are rows and models are columns
gic_df = gic_df.T
gic_df

/var/folders/ns/zy_pbb_s3591fwh2_q7z6g0h0000gn/T/ipykernel_21275/3525529518.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  gic_df[graph_name] = model_gics
/var/folders/ns/zy_pbb_s3591fwh2_q7z6g0h0000gn/T/ipykernel_21275/3525529518.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  gic_df[graph_name] = model_gics
/var/folders/ns/zy_pbb_s3591fwh2_q7z6g0h0000gn/T/ipykernel_21275/3525529518.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor

model,LG,ER,BA,WS,GRG
134208594.edges,0.215938,0.408830,0.553002,1.485660,2.791326
23503181.edges,0.795880,0.712607,0.594682,1.359712,4.282978
28628055.edges,0.787652,0.636669,0.486718,1.248200,6.476412
5773532.edges,0.218005,0.599033,0.411241,1.117310,8.528437
54226675.edges,0.897812,0.883680,0.269791,1.302425,6.202168
...,...,...,...,...,...
17658786.edges,0.193770,0.253430,0.642195,1.731841,1.857099
926981.edges,0.386912,0.383959,0.629418,1.436649,4.191461
15208246.edges,0.099026,0.158526,0.679151,1.750017,1.714391
77404713.edges,0.615801,1.909831,0.521340,1.021292,8.634934


In [11]:
# Create a dictionary to store rankings for each graph
rankings = {}

# Go through each df in dfs
for df in dfs:
    graph_name = df['graph_filename'].iloc[0]  # Get graph filename
    
    # Sort models by GIC value (excluding 'Original' which has NaN GIC)
    model_ranks = df[df['model'] != 'Original'].sort_values('gic_value')['model'].tolist()
    
    # Create rankings dictionary for this graph (1 is best since lowest GIC is best)
    rankings[graph_name] = {
        model: rank+1 for rank, model in enumerate(model_ranks)
    }
    
    # Add nodes and edges from Original model
    original_row = df[df['model'] == 'Original'].iloc[0]
    rankings[graph_name]['nodes'] = original_row['nodes']
    rankings[graph_name]['edges'] = original_row['edges']

# Get unique models (excluding 'Original')
models = sorted(list(set([model for df in dfs for model in df['model'].unique() if model != 'Original'])))

# Create DataFrame with rankings
ranking_df = pd.DataFrame(rankings).T

# Ensure all models are present as columns
for model in models:
    if model not in ranking_df.columns:
        ranking_df[model] = np.nan

# Keep only the model columns in a consistent order, plus nodes and edges
ranking_df = ranking_df[models + ['nodes', 'edges']]

print("Rankings by GIC value (lower is better):")
display(ranking_df)


Rankings by GIC value (lower is better):


,BA,ER,GRG,LG,WS,nodes,edges
134208594.edges,3,2,5,1,4,100,1677
23503181.edges,1,2,5,3,4,101,1455
28628055.edges,1,2,5,3,4,101,1105
5773532.edges,2,3,5,1,4,101,841
54226675.edges,1,2,5,3,4,102,953
...,...,...,...,...,...,...,...
17658786.edges,3,2,5,1,4,202,5513
926981.edges,3,1,5,2,4,202,3676
15208246.edges,3,2,4,1,5,202,5864
77404713.edges,1,4,5,2,3,202,1801


In [12]:
ranking_df.mean(axis=0)

BA          2.131222
ER          2.721719
GRG         4.957014
LG          2.104072
WS          3.085973
nodes     153.977376
edges    2062.006787
dtype: float64

In [13]:
ranking_df['nodes'].describe()

count    442.000000
mean     153.977376
std       28.868918
min      100.000000
25%      129.250000
50%      153.000000
75%      179.750000
max      202.000000
Name: nodes, dtype: float64